In [1]:
import autogen
from typing import Annotated
import datetime


In [2]:
def get_weather(location: Annotated[str,"the location"]) ->str:
    if location == "Florida":
        return "It's hot in Florida!"
    elif location == "Maine":
        return "It's cold in Maine"
    else:
        return f"I don't know this place {location}"

In [4]:
def get_time(timezone: Annotated[str, "The timezone we are in"]) -> str:
    now = datetime.datetime.now()

    # Get the time with timezone information
    current_time = now.strftime("%Y-%m-%d %H:%M:%S %Z")
    return f"Current time in your {timezone}: {current_time}"


In [ ]:
# Let's first define the assistant agent that suggests tool calls.
assistant = autogen.ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
                   "Return 'TERMINATE' when the task is done.",
    llm_config={
        "config_list": [
            {
        "model": "gemini-pro",
        "api_key": "",
        "api_type": "google"
    }
    
        ]
    }
)


In [11]:
# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = autogen.ConversableAgent(
    name="User",
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

In [12]:
# Register the tool signature with the assistant agent.
assistant.register_for_llm(name="get_weather", description="Get the current weather for a specific location")(get_weather)
assistant.register_for_llm(name="get_time", description="The IANA time zone name, e.g. America/Los_Angeles")(get_time)


<function __main__.get_time(timezone: typing.Annotated[str, 'The timezone we are in']) -> str>

In [13]:
# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="get_weather")(get_weather)
user_proxy.register_for_execution(name="get_time")(get_time)

<function __main__.get_time(timezone: typing.Annotated[str, 'The timezone we are in']) -> str>

In [14]:
user_proxy.initiate_chat(
    assistant,
    message="What time is it in Florida?"
)

User (to Assistant):

What time is it in Florida?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (123): get_time *****
Arguments: 
{"timezone": "America/New_York"}
***********************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_time...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (123) *****
Current time in your America/New_York: 2024-12-09 20:40:08 
********************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (440): get_weather *****
Arguments: 
{"location": "Florida"}
**************************************************

--------------------------------------------------------------------------------

>>>

ChatResult(chat_id=None, chat_history=[{'content': 'What time is it in Florida?', 'role': 'assistant', 'name': 'User'}, {'tool_calls': [{'id': '123', 'function': {'arguments': '{"timezone": "America/New_York"}', 'name': 'get_time'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': 'Current time in your America/New_York: 2024-12-09 20:40:08 ', 'tool_responses': [{'tool_call_id': '123', 'role': 'tool', 'content': 'Current time in your America/New_York: 2024-12-09 20:40:08 ', 'name': 'get_time'}], 'role': 'tool', 'name': 'User'}, {'tool_calls': [{'id': '440', 'function': {'arguments': '{"location": "Florida"}', 'name': 'get_weather'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'content': "It's hot in Florida!", 'tool_responses': [{'tool_call_id': '440', 'role': 'tool', 'content': "It's hot in Florida!", 'name': 'get_weather'}], 'role': 'tool', 'name': 'User'}, {'tool_calls': [{'id': '789', 'function': {'arguments': '{"timezone": "America/Los_Ange